File: population_simulation.py

Author: Shai Sussman

Date: 26 May 2023

Description: This script initates the dashboard

visit http://127.0.0.1:8050/ in your web browser.

In [ ]:
#Installing specific packages.
!venv/bin/python -m pip install dash==2.10.0
!venv/bin/python -m pip install jupyter_dash==0.4.2
!venv/bin/python -m pip install geopandas==0.13.0
!venv/bin/python -m pip install dash-leaflet==0.1.23
!venv/bin/python -m pip install dash-extensions==0.0.65
!venv/bin/python -m pip install dash_daq==0.5.0
!/Users/shai/anaconda3/bin/python -m pip install dash-bootstrap-components==1.4.1

In [9]:
import requests
from dash import Dash, html, dcc, Input, Output
from dash_extensions.javascript import  assign
import dash_daq as daq
import dash_bootstrap_components as dbc
from plotly.subplots import make_subplots
import dash_leaflet.express as dlx
import plotly.express as px
import plotly.graph_objects as go
import dash_leaflet as dl
import pandas as pd
import numpy as np
import json
import geopandas as gpd
from jupyter_dash import JupyterDash
import datetime
import requests
from dash_extensions.javascript import Namespace
ns = Namespace("someNamespace", "someSubNamespace")


In [31]:
# Pandas configuration
pd.options.display.max_columns=100

# Globals and CONSTANT
contextual_width_global = 1200
contextual_height_global = 275

status_graph_categories = ['Building before','Construction','Building after']
status_graph_color = ['rgba(230,230,230, 0.65)','rgba(249,199,15, 0.65)','rgba(68,115,197, 0.65)']
status_grpah_color_discrete_map = dict(zip(status_graph_categories,status_graph_color))

# Use colab
USECOLAB = False

if USECOLAB:
    PATH_TO_POUP = '/content/assets/popup.js'
else:
    PATH_TO_POUP = 'assets/popup.js'


# Loading Files

In [11]:

#Statistical Stats
statisticalStatsGdf = gpd.read_file('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/statistical_tract_4326.geojson')
statisticalStatsJson = json.loads(statisticalStatsGdf.to_json())
#building simulation
simulatedBldgsGdf = gpd.read_file('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/buildings_for_dashboard_4326.geojson')
simulatedBldgsJson = json.loads(simulatedBldgsGdf.to_json())
simulatedBldgsGdf['start_date'] = pd.to_datetime(simulatedBldgsGdf['start_date'])
simulatedBldgsGdf['end_date'] = pd.to_datetime(simulatedBldgsGdf['end_date'])
#agents track
agents_track_status = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/agents_track_status.csv')
agents_track_status.drop(columns='Unnamed: 0',inplace=True)

agents_stat_summary_by_year = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/yearly_stats_for_dashboard.csv')

year_ranges = agents_stat_summary_by_year['year']

color_labels = pd.read_excel(
    'https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/color_labels.xlsx')
colorDict = dict(zip(color_labels['label'], color_labels['colors_']))


attribution = '© OpenStreetMap contributors, © CARTO'
cartoUrl = 'http://basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png'


In [13]:
if USECOLAB:
    url = 'https://shai2u.github.io/demographic_estimation_dashboard_article/BatYam_maplibre/map.html'
    r = requests.get(url, allow_redirects=True)
    open('/content/map.html', 'wb').write(r.content)


## variables and parameters

# Agents Non Agregated preprocessing

In [14]:
incomeDict = {9000:'Low',19500:'Medium', 1000000:'High'}
agents_track_status['income_cat'] = pd.cut(agents_track_status['income'], [0,9000,19500,1000000],right=True, labels=['Low','Medium','High'],ordered=True)

agents_track_status.loc[agents_track_status['rent']==1,'rent_own']='Rent'
agents_track_status.loc[agents_track_status['rent']==0,'rent_own']='Own'

## Pre-processing

In [15]:

agents_stat_summary_by_year['New Comers'] = agents_stat_summary_by_year['New Comers_income_low'] + agents_stat_summary_by_year['New Comers_income_medium'] +  + agents_stat_summary_by_year['New Comers_income_high']
agents_stat_summary_by_year['stay'] = agents_stat_summary_by_year['stay_income_low'] + agents_stat_summary_by_year['stay_income_medium'] +  + agents_stat_summary_by_year['stay_income_high']
agents_stat_summary_by_year['total_pop'] = agents_stat_summary_by_year['New Comers'] + agents_stat_summary_by_year['stay']
agents_stat_summary_by_year['New Comers_income_low_ratio'] = agents_stat_summary_by_year['New Comers_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_medium_ratio'] = agents_stat_summary_by_year['New Comers_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_high_ratio'] = agents_stat_summary_by_year['New Comers_income_high']/agents_stat_summary_by_year['total_pop']

agents_stat_summary_by_year['stay_income_low_ratio'] = agents_stat_summary_by_year['stay_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_medium_ratio'] = agents_stat_summary_by_year['stay_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_high_ratio'] = agents_stat_summary_by_year['stay_income_high']/agents_stat_summary_by_year['total_pop']


# Graphs

# status graph

In [16]:
# Generate count status graph
def get_status_graph(bldg_status):
  #"status"
  fig = px.bar(bldg_status, x='status', y="count",width=600,height=250,color='status',color_discrete_map=status_grpah_color_discrete_map,template='plotly_white',category_orders={'status':['Building before', 'Construction', 'Building after']})
  #, pattern_shape_sequence=["+"]
  fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0,"pad":0},
  showlegend=False,

  # font=dict(
  #     size=15,
  # )
  )
  fig.update_yaxes(automargin=True)
  fig.update_yaxes(range=[0, 70])
    # Add images
  fig.add_layout_image(
      dict(
          source="https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/graphics/Status_Graph.jpg",
           xref="paper", yref="paper",
           x=0, y=0,
           sizex=1, sizey=1,
           xanchor="left", yanchor="bottom",
           layer="below"))
  fig.update_yaxes(showticklabels=False,visible=False)
  fig.update_xaxes(showticklabels=False,visible=False)
  fig.update_traces(
    #marker=dict(line_color="grey", pattern_fillmode="replace") #excellent!!
    marker=dict(line_color="black")
  )

          
  return fig


# Dot Matrix

In [17]:

def exportFigure_sq(list_,date_,MatrixPlot):

    # Use column names of df for the different parameters x, y, color, ...
    MatrixPlot['Who'] = list_
    # MatrixPlot['size_'] = 4
    MatrixPlot['squre_'] = 'square'
    #,symbol='squre_',symbol_map = {'square':'square'}
    fig = px.scatter(MatrixPlot, x="x", y="y", color="Who",
                     title=f'Staying vs Leaving Snapshot {date_}',
                     labels={"Who":"Legend"}, # customize axis label
                     template='simple_white',color_discrete_map={
                         "Staying": "firebrick",
                         "New Comers": "royalblue",
                         "Future Units": "hsv(0,0%,95%)"})
    #orientation="h"
    fig.update_layout(width=580,height=450,margin=dict(l=0, r=0, t=30, b=0),legend=dict( yanchor="top",y=0.95,xanchor="left",x=0.01, font = dict(size = 15)))
    fig.update_yaxes(showticklabels=False,visible=False)
    fig.update_xaxes(showticklabels=False,visible=False)
    fig.update_traces(marker=dict(size=5.5))

    return fig

In [18]:

#fig.write_image(f'matrixPlot_{ticNumber}.png')

#create an empty dataset with agent coutn
num_of_agents = 3481
MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
#Number of Columns choose:
numOfColumns=59
numOfRows = 59
# set the columns to display the column number
MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
#prepare the y column (row number)
y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
temp = []
for i in range(0,numOfRows):
    temp +=y[i]
y = temp
MatrixPlot['y'] = y

agents_time_new_Comers_stay = agents_stat_summary_by_year[['year','New Comers','stay']].fillna(0)
agents_time_new_Comers_stay['emptyPalces'] = num_of_agents -(agents_time_new_Comers_stay['stay'] +agents_time_new_Comers_stay['New Comers'])

list_ = ['Staying']* agents_time_new_Comers_stay.loc[20,'stay'].astype(int)
list_ = list_ + ['New Comers']*agents_time_new_Comers_stay.loc[20,'New Comers'].astype(int)
list_ = list_ + ['Future Units']*agents_time_new_Comers_stay.loc[20,'emptyPalces'].astype(int)
dotMAtrixFig1 = exportFigure_sq(list_,('temp'),MatrixPlot)



# Contextual Time Series

## renters_owners_graph

In [19]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
def renters_owners_graph(year_q,q_date):
  rent_own_new_stay = agents_stat_summary_by_year[['year','New Comers_rent','New Comers_own','stay_rent','stay_own']].copy().fillna(0)
  end_index = rent_own_new_stay[rent_own_new_stay['year']==year_q].index.values[0]
  selected_indexes = range(0,end_index+1)
  rent_own_new_stayselected = rent_own_new_stay[rent_own_new_stay.index.isin(selected_indexes)]

  fig = go.Figure()

  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=rent_own_new_stayselected['New Comers_rent'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Renters",
      mode="lines",
      line=dict(color='royalblue', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=rent_own_new_stayselected['New Comers_own'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Owners",
      mode="lines",
      line=dict(color='royalblue', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=rent_own_new_stayselected['stay_rent'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Renters",
      mode="lines",
      line=dict(color='firebrick', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=rent_own_new_stayselected['stay_own'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Owners",
      mode="lines",
      line=dict(color='firebrick', width=5)
  ))

  fig.add_trace(go.Scatter(x=[q_date, q_date], y=[0,2100],
                name=q_date,
                legendgroup="Reference",
                legendgrouptitle_text="Reference",
                mode="lines",
                line=dict(color="LightSeaGreen", width=2,dash="dashdot")))
  

  fig.update_layout(title="Staying Leaving and Owenrship",template='plotly_white',yaxis = {'title' : "Absolute Numbers"},margin={"r":0,"t":35,"l":0,"b":35,"pad":0},
                    width=contextual_width_global,height=contextual_height_global,legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
  fig.update_layout(hovermode="x unified")
  fig.update_xaxes(range = [0,len(year_ranges)])
  fig.update_yaxes(range = [0,2100])
  return fig

# set range
renters_owners_fig = renters_owners_graph('2015 Q1','2015 Q1')

## apartment_graph

In [20]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
def apartment_graph(year_q,q_date):
  df_less_columns = agents_stat_summary_by_year[['year','New Comers_apartment_size_q1','New Comers_apartment_size_q2','New Comers_apartment_size_q3','stay_apartment_size_q1','stay_apartment_size_q2','stay_apartment_size_q3']].copy()
  end_index = df_less_columns[df_less_columns['year']==year_q].index.values[0]
  selected_indexes = range(0,end_index+1)
  df_for_graph = df_less_columns[df_less_columns.index.isin(selected_indexes)]

  fig = go.Figure()


  #start modification



  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_apartment_size_q1'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Apartment Size Q1 (25%)",
      mode="lines",
      line=dict(color='royalblue', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_apartment_size_q2'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Apartment Size Q2 (50%)",
      mode="lines",
      line=dict(color='royalblue', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_apartment_size_q3'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Apartment Size Q3 (75%)",
      mode="lines",
      line=dict(color='royalblue', width=4)
      
  ))

  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_apartment_size_q1'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Apartment Size Q1 (25%)",
      mode="lines",
      line=dict(color='firebrick', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_apartment_size_q2'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Apartment Size Q2 (50%)",
      mode="lines",
      line=dict(color='firebrick', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_apartment_size_q3'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Apartment Size Q3 (75%)",
      mode="lines",
      line=dict(color='firebrick', width=4)
      
  ))

  fig.add_trace(go.Scatter(x=[q_date, q_date], y=[50,140],
              name=q_date,
              legendgroup="Reference",
              legendgrouptitle_text="Reference",
              mode="lines",
              line=dict(color="LightSeaGreen", width=2,dash="dashdot")))

  # fig.update_layout(title="Apartment Size  (q1-q3) vs Staying or New Comers",template='plotly_white',yaxis = {'title' : "m<sup>2</sup>"})
  
  fig.update_layout(title="Apartment Size quarter distribution for Staying vs New Comers",template='plotly_white',yaxis = {'title' : "m<sup>2</sup>"},margin={"r":0,"t":35,"l":0,"b":35,"pad":0},
                    width=contextual_width_global,height=contextual_height_global,legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))

  fig.update_layout(hovermode="x unified")

  #end modification

  fig.update_xaxes(range = [0,len(year_ranges)])
  fig.update_yaxes(range = [50,160])
  return fig

# set range
#renters_owners_fig = renters_owners_graph('2015 Q1','2015 Q1')

## change_age_distribution_graph

In [21]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
def change_age_distribution_graph(year_q,q_date):
  df_less_columns = agents_stat_summary_by_year[['year','New Comers_age_q1','New Comers_age_q2','New Comers_age_q3','stay_age_q1','stay_age_q2','stay_age_q3']].copy()
  end_index = df_less_columns[df_less_columns['year']==year_q].index.values[0]
  selected_indexes = range(0,end_index+1)
  df_for_graph = df_less_columns[df_less_columns.index.isin(selected_indexes)]

  fig = go.Figure()


  # start modification


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_age_q1'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Average age in q1 (25%)",
      mode="lines",
      line=dict(color='royalblue', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_age_q2'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Average age in q2 (50%)",
      mode="lines",
      line=dict(color='royalblue', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_age_q3'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Average age in q3 (75%)",
      mode="lines",
      line=dict(color='royalblue', width=4)
      
  ))

  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_age_q1'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Average age in q1 (25%)",
      mode="lines",
      line=dict(color='firebrick', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_age_q2'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Average age in q2 (50%)",
      mode="lines",
      line=dict(color='firebrick', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_age_q3'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Average age in q2 (75%)",
      mode="lines",
      line=dict(color='firebrick', width=4)
      
  ))

  

  fig.add_trace(go.Scatter(x=[q_date, q_date], y=[20,100],
              name=q_date,
              legendgroup="Reference",
              legendgrouptitle_text="Reference",
              mode="lines",
              line=dict(color="LightSeaGreen", width=2,dash="dashdot")))
  
  fig.update_layout(title="Age quarter distribution for Staying vs New Comers",template='plotly_white',yaxis = {'title' : "Average Age"},margin={"r":0,"t":35,"l":0,"b":35,"pad":0},
                    width=contextual_width_global,height=contextual_height_global,legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
  fig.update_layout(hovermode="x unified")

  fig.update_xaxes(range = [0,len(year_ranges)])
  fig.update_yaxes(range = [20,110])
  #end modification

  return fig

# set range
graph = change_age_distribution_graph('2025 Q1','2015 Q1')

## income_distribution_graph

In [22]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
def income_distribution_graph(year_q,q_date):
  df_less_columns = agents_stat_summary_by_year[['year','New Comers_income_q1','New Comers_income_q2','New Comers_income_q3','stay_income_q1','stay_income_q2','stay_income_q3']].copy()
  end_index = df_less_columns[df_less_columns['year']==year_q].index.values[0]
  selected_indexes = range(0,end_index+1)
  df_for_graph = df_less_columns[df_less_columns.index.isin(selected_indexes)]

  fig = go.Figure()




  #start modification



  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_income_q1'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Income Q1 (25%)",
      mode="lines",
      line=dict(color='royalblue', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_income_q2'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Income Q2 (50%)",
      mode="lines",
      line=dict(color='royalblue', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_income_q3'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Income Q3 (75%)",
      mode="lines",
      line=dict(color='royalblue', width=4)
      
  ))

  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_income_q1'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Icome Q1 (25%)",
      mode="lines",
      line=dict(color='firebrick', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_income_q2'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Income Q2 (50%)",
      mode="lines",
      line=dict(color='firebrick', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_income_q3'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Income Q3 (75%)",
      mode="lines",
      line=dict(color='firebrick', width=4)
      
  ))



  fig.add_trace(go.Scatter(x=[q_date, q_date], y=[0,40000],
              name=q_date,
              legendgroup="Reference",
              legendgrouptitle_text="Reference",
              mode="lines",
              line=dict(color="LightSeaGreen", width=2,dash="dashdot")))
  

  fig.update_layout(title="Income quarter distribution for Staying vs New Comers",template='plotly_white',yaxis = {'title' : "Inocme (New Israeli Shekels)"},margin={"r":0,"t":35,"l":0,"b":35,"pad":0},
                    width=contextual_width_global,height=contextual_height_global,legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
  fig.update_layout(hovermode="x unified")


  fig.update_xaxes(range = [0,len(year_ranges)])
  fig.update_yaxes(range = [0,40000])

  #end modification

  return fig

# set range
graph = income_distribution_graph('2025 Q1','2015 Q1')

## income_category_graph

In [23]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
def income_category_graph(year_q,q_date):
  df_less_columns = agents_stat_summary_by_year[['year','New Comers_income_low_ratio','New Comers_income_medium_ratio','New Comers_income_high_ratio','stay_income_low_ratio','stay_income_medium_ratio','stay_income_high_ratio']].copy()
  end_index = df_less_columns[df_less_columns['year']==year_q].index.values[0]
  selected_indexes = range(0,end_index+1)
  df_for_graph = df_less_columns[df_less_columns.index.isin(selected_indexes)]

  fig = go.Figure()


  #start modification



  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_income_low_ratio'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Low Inocme",
      mode="lines",
      line=dict(color='royalblue', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_income_medium_ratio'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Medium Income",
      mode="lines",
      line=dict(color='royalblue', width=3)
  ))



  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_income_high_ratio'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="High Income",
      mode="lines",
      line=dict(color='royalblue', width=5)
  ))

  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_income_low_ratio'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Low Income",
      mode="lines",
      line=dict(color='firebrick', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_income_medium_ratio'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Medium Income",
      mode="lines",
      line=dict(color='firebrick', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_income_high_ratio'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="High Income",
      mode="lines",
      line=dict(color='firebrick', width=5)
  ))




  fig.add_trace(go.Scatter(x=[q_date, q_date], y=[0,1],
              name=q_date,
              legendgroup="Reference",
              legendgrouptitle_text="Reference",
              mode="lines",
              line=dict(color="LightSeaGreen", width=2,dash="dashdot")))
  

  fig.update_layout(title="Income by class for Staying vs New Comers",template='plotly_white',yaxis = {'title' : "ratio"},margin={"r":0,"t":35,"l":0,"b":35,"pad":0},
                    width=contextual_width_global,height=contextual_height_global,legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
  fig.update_layout(hovermode="x unified")


  fig.update_xaxes(range = [0,len(year_ranges)])
  fig.update_yaxes(range = [0,1])


  #end modification

  return fig

# set range
graph = income_category_graph('2025 Q1','2015 Q1')

# Construction

In [24]:



def construction_typo_graph_generator(construction_typo_v,construction_typo_d):

  try:
    v_a = construction_typo_v[construction_typo_v['project_ty']==1]['count'].values[0]
  except:
    v_a = 0
  try:
    v_r = construction_typo_v[construction_typo_v['project_ty']==2]['count'].values[0]
  except:
    v_r = 0
  try:
    v_rr = construction_typo_v[construction_typo_v['project_ty']==3]['count'].values[0]
  except:
    v_rr = 0

  try:
    d_a = construction_typo_d[construction_typo_d['project_ty']==1]['count'].values[0]
  except:
    d_a = 0
  try:
    d_r = construction_typo_d[construction_typo_d['project_ty']==2]['count'].values[0]
  except:
    d_r = 0
  try:
    d_rr = construction_typo_d[construction_typo_d['project_ty']==3]['count'].values[0]
  except:
    d_rr = 0
  fig = go.Figure()

  fig.add_trace(go.Indicator(
      mode = "number+delta",
      value = v_a,
      domain = {'x': [0.06, 0.25], 'y': [0.7, 0.85]},
      number_font_color="#2052a7",
      
      delta = {'reference': d_a, 'relative': True,'valueformat':'.2%'}))

  fig.add_trace(go.Indicator(
      mode = "number+delta",
      value = v_r,
      domain = {'x': [0.3, 0.5], 'y': [0.7, 0.85]},
      number_font_color="#4c84cb",
      delta = {'reference': d_r, 'relative': True,'valueformat':'.2%'}))

  fig.add_trace(go.Indicator(
      mode = "number+delta",
      value = v_rr,
      domain = {'x': [0.6, 0.80], 'y': [0.7, 0.85]},
      number_font_color="#87b1eb",
      delta = {'reference': d_rr, 'relative': True,'valueformat':'.2%'}))


  # Add images
  fig.add_layout_image(
          dict(
              source="https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/graphics/Building_Typology_png.png",
              xref="paper", yref="paper",
              x=0, y=0,
              sizex=1, sizey=1,
              xanchor="left", yanchor="bottom",
              layer="below")
  )

  fig.update_layout(template="plotly_white",margin={"r":0,"t":0,"l":0,"b":0,"pad":0},width=600,height=250)
  return fig


In [25]:
d = pd.to_datetime('2015-01-01')
d2 = d - datetime.timedelta(days=180)
bldgs1 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)

bldgs_constr = bldgs1[bldgs1['status']=='Construction'].reset_index()
#Add all columns
# bldg_status = bldgs1['status'].value_counts().to_frame().reset_index()
# bldg_status.rename(columns={'index':'status','status':'count'},inplace=True)

# if len(bldg_status)<3:
#   if len(bldg_status[bldg_status['status'].isin(['Building before'])])==0:
#     bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building before'],'count':[0]})]).reset_index(drop=True)
#   if len(bldg_status[bldg_status['status'].isin(['Construction'])])==0:
#     bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Construction'],'count':[0]})]).reset_index(drop=True)
#   if len(bldg_status[bldg_status['status'].isin(['Building after'])])==0:
#     bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building after'],'count':[0]})]).reset_index(drop=True)

construction_typo_v = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
construction_typo_v.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
bldgs2 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d2 ) & (simulatedBldgsGdf['end_date']> d2 )].copy().reset_index(drop=True)

bldgs_constr = bldgs2[bldgs2['status']=='Construction'].reset_index()

construction_typo_d = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
construction_typo_d.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
construction_typo_graph = construction_typo_graph_generator(construction_typo_v,construction_typo_d)

# Suburst

In [26]:
def demographic_sunburst(year_q,agents_stay_age_income,colorDict):
  title_ = f'{year_q} : Population composition snapshot'
  # color_discrete_map = color_group_map_
  fig = px.sunburst(agents_stay_age_income, path=['Stay or leave','Age group','Income category'], title=title_)

  labels_text = fig.data[0].labels.tolist()
  colorLabels = tuple(colorDict[item] for item in labels_text)
  fig.data[0].marker.colors = colorLabels
  fig.update_traces(textinfo="label+percent entry")
  fig.update_layout(showlegend=False, margin=dict(l=0, r=25, t=50, b=0), width=550, height=450)
  fig.add_layout_image(
    dict(
        source="https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/graphics/legend.png",
          xref="paper", yref="paper",
          x=0.15, y=0,
          sizex=1, sizey=1,
          xanchor="left", yanchor="bottom",
          layer="below"))
  return fig


In [27]:

year_makrs = [year for year in np.arange(2015,2031,0.5)]
years_with_q2_makrs = []
for year in year_makrs:
    d = pd.to_datetime(f'{int(year)}-07-01')
    if year % 1 == 0:
        d = pd.to_datetime(f'{int(year)}-01-01')
    years_with_q2_makrs.append(d)
    
d = pd.to_datetime('2015-01-01')
bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
bldgs1 = bldgs.copy()
agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

age_grown1 = d.year-years_with_q2_makrs[0].year
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})

In [28]:
population_sunburst_graph_init = demographic_sunburst(2019,agents_stay_age_income,colorDict)

# Bubble Figure

In [29]:
def bubble_age_income_stay_time(StayAgeIncome_ReftedDate,NewComersAgeIncome_RefDate,StayAgeIncome_SelectedDate,NewComersAgeIncome_SelectedDate,ref_date_,date_):
  #Reference plot
  fig = make_subplots(rows=1, cols=2,subplot_titles=(f"Reference Date {ref_date_}", f"Current Date {date_}"))
  fig.add_trace(
      go.Scatter(x=NewComersAgeIncome_RefDate['Income category'], y=NewComersAgeIncome_RefDate['Age group'], mode="markers",name='New Comers',marker=dict(size=NewComersAgeIncome_RefDate['ratio'] * 250,opacity=0.5,color='blue'),showlegend=True,hovertemplate='Households %{text}',text = list(NewComersAgeIncome_RefDate['units'])),

      row=1, col=1
  )
  fig.add_trace(
      go.Scatter(x=StayAgeIncome_ReftedDate['Income category'], y=StayAgeIncome_ReftedDate['Age group'], mode="markers",name='Staying',marker=dict(size=StayAgeIncome_ReftedDate['ratio'] * 250,opacity=0.5,color='red'),showlegend=True,hovertemplate='Households %{text}',text = list(StayAgeIncome_ReftedDate['units'])),
      row=1, col=1
  )
  
  fig.add_trace(
      go.Scatter(x=NewComersAgeIncome_SelectedDate['Income category'], y=NewComersAgeIncome_SelectedDate['Age group'], mode="markers",name='New Comers',marker=dict(size=NewComersAgeIncome_SelectedDate['ratio'] * 250,opacity=0.5,color='blue'),showlegend=True,hovertemplate='Households %{text}',text = list(NewComersAgeIncome_SelectedDate['units'])),
      row=1, col=2
  )
  fig.add_trace(
      go.Scatter(x=StayAgeIncome_SelectedDate['Income category'], y=StayAgeIncome_SelectedDate['Age group'], mode="markers",name='Staying',marker=dict(size=StayAgeIncome_SelectedDate['ratio'] * 250 ,opacity=0.5,color='red'),showlegend=True,hovertemplate='Households %{text}',text = list(StayAgeIncome_SelectedDate['units'])),
      row=1, col=2
  )
  # text = list(StayAgeIncome_SelectedDate['units'])
  # hover_text = '%{text}'


  #hovertemplate="name: %{y}%{x}<br>number: %{marker.symbol}<extra></extra>")


  fig.update_layout(height=450, width=450, title_text="Side By Side Subplots", margin=dict(l=0, r=0, t=50, b=0),legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01),template='plotly_white')
  return fig

# get date from function
d = pd.to_datetime('2015-01-01')
bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
bldgs1 = bldgs.copy()

agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

#age the agents
age_grown1 = d.year-years_with_q2_makrs[0].year
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

#set categories to agents ages
agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
#population_sunburst_graph = demographic_sunburst(2019,agents_stay_age_income,colorDict)
agents_stay_age_income['units'] = 1
agents_stay_age_income_count = agents_stay_age_income.groupby(['Stay or leave','Age group','Income category']).agg({'units':'count'}).reset_index()
new_comers_age_income_count = agents_stay_age_income_count[agents_stay_age_income_count['Stay or leave']=='New Comers'].reset_index(drop=True)
stay_age_income_count = agents_stay_age_income_count[agents_stay_age_income_count['Stay or leave']=='stay'].reset_index(drop=True)
total = new_comers_age_income_count['units'].sum()+ stay_age_income_count['units'].sum()
new_comers_age_income_count['ratio'] = new_comers_age_income_count['units']/1100
stay_age_income_count['ratio'] = stay_age_income_count['units']/1100

d_ref = pd.to_datetime('2015-01-01')
bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d_ref ) & (simulatedBldgsGdf['end_date']> d_ref )].copy().reset_index(drop=True)
bldgs1 = bldgs.copy()

agents_synced_buildings_ref_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
agents_synced_buildings_ref_date_stay_new = agents_synced_buildings_ref_date[agents_synced_buildings_ref_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

#age the agents
age_grown1 = d_ref.year-years_with_q2_makrs[0].year
agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='stay','age'] + age_grown1
agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d_ref.year - p['start_date'].year),axis=1)

#set categories to agents ages
agents_synced_buildings_ref_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_ref_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
agents_stay_age_income_ref =agents_synced_buildings_ref_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
#population_sunburst_graph = demographic_sunburst(2019,agents_stay_age_income,colorDict)
agents_stay_age_income_ref['units'] = 1
agents_stay_age_income_ref = agents_stay_age_income_ref.groupby(['Stay or leave','Age group','Income category']).agg({'units':'count'}).reset_index()
new_comers_age_income_count_ref = agents_stay_age_income_ref[agents_stay_age_income_ref['Stay or leave']=='New Comers'].reset_index(drop=True)
stay_age_income_count_ref = agents_stay_age_income_ref[agents_stay_age_income_ref['Stay or leave']=='stay'].reset_index(drop=True)
total_ref = new_comers_age_income_count_ref['units'].sum()+ stay_age_income_count_ref['units'].sum()
new_comers_age_income_count_ref['ratio'] = new_comers_age_income_count_ref['units']/1100
stay_age_income_count_ref['ratio'] = stay_age_income_count_ref['units']/1100
bubble_age_income = bubble_age_income_stay_time(stay_age_income_count_ref,new_comers_age_income_count_ref,stay_age_income_count,new_comers_age_income_count,'2015 Q1','2015 Q1')


# Dash

In [30]:

def prepare_dot_matrix(q_date_for_dot_matrix):
    # #agents_stat_summary_by_year = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/yearly_stats_for_dashboard.csv')

    # agents_stat_summary_by_year['New Comers'] = agents_stat_summary_by_year['New Comers_income_low'] + agents_stat_summary_by_year['New Comers_income_medium'] +  + agents_stat_summary_by_year['New Comers_income_high']
    # agents_stat_summary_by_year['stay'] = agents_stat_summary_by_year['stay_income_low'] + agents_stat_summary_by_year['stay_income_medium'] +  + agents_stat_summary_by_year['stay_income_high']
    # agents_stat_summary_by_year['total_pop'] = agents_stat_summary_by_year['New Comers'] + agents_stat_summary_by_year['stay']
    # agents_stat_summary_by_year['New Comers_income_low_ratio'] = agents_stat_summary_by_year['New Comers_income_low']/agents_stat_summary_by_year['total_pop']
    # agents_stat_summary_by_year['New Comers_income_medium_ratio'] = agents_stat_summary_by_year['New Comers_income_medium']/agents_stat_summary_by_year['total_pop']
    # agents_stat_summary_by_year['New Comers_income_high_ratio'] = agents_stat_summary_by_year['New Comers_income_high']/agents_stat_summary_by_year['total_pop']

    # agents_stat_summary_by_year['stay_income_low_ratio'] = agents_stat_summary_by_year['stay_income_low']/agents_stat_summary_by_year['total_pop']
    # agents_stat_summary_by_year['stay_income_medium_ratio'] = agents_stat_summary_by_year['stay_income_medium']/agents_stat_summary_by_year['total_pop']
    # agents_stat_summary_by_year['stay_income_high_ratio'] = agents_stat_summary_by_year['stay_income_high']/agents_stat_summary_by_year['total_pop']

    #create an empty dataset with agent coutn
    num_of_agents = 3481
    MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
    #Number of Columns choose:
    numOfColumns=59
    numOfRows = 59
    # set the columns to display the column number
    MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
    #prepare the y column (row number)
    y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
    temp = []
    for i in range(0,numOfRows):
        temp +=y[i]
    y = temp
    MatrixPlot['y'] = y
    agents_stat_short = agents_stat_summary_by_year[['year','New Comers','stay']].copy().fillna(0)
    agents_stat_short['emptyPalces'] =agents_stat_short.apply(lambda p:num_of_agents-(p['stay']+p['New Comers']), axis=1)
    stay_count = int(agents_stat_short.loc[agents_stat_short['year']==q_date_for_dot_matrix,'stay'].values[0])
    new_comers = int(agents_stat_short.loc[agents_stat_short['year']==q_date_for_dot_matrix,'New Comers'].values[0])
    empty_space = int(agents_stat_short.loc[agents_stat_short['year']==q_date_for_dot_matrix,'emptyPalces'].values[0])
    list_ = ['Staying']* stay_count
    list_ = list_ + ['New Comers'] * new_comers
    list_ = list_ + ['Future Units'] * empty_space
    dotMAtrixFig = exportFigure_sq(list_,(q_date_for_dot_matrix),MatrixPlot)
    return dotMAtrixFig
    #dotMAtrixFig = dotMAtrixFig1

line_style = dict(weight=2, opacity=1, color='blue', fillOpacity=0,dashArray="10 10")

classes = ['Building before', 'Construction', 'Building after']
colorscale = ['#808080', '#F9C70F', '#4473C5']
style = dict(weight=2, opacity=1, color='grey', fillOpacity=0.7)
colorbar = dlx.categorical_colorbar(categories=classes, colorscale=colorscale, width=300, height=30, position="bottomright")

#globals 
contextual_width_global = 1200
contextual_height_global = 275
map_height = '600px'
lower_fig_height = 450

style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.props.hideout;  // get props from hideout
    const value = feature.properties[colorProp];  // get value the determines the color
    for (let i = 0; i < classes.length; ++i) {
        if (value == classes[i]) {
            style.fillColor = colorscale[i];  // set the fill color according to the class
        }
    }
    return style;
}""")


#dl.Map()

#geojson = dl.GeoJSON(data=data, options=dict(onEachFeature=ns("bindPopup")))

mapObj = dl.Map([dl.TileLayer(url=cartoUrl, maxZoom=20, attribution=attribution),dl.GeoJSON(data=statisticalStatsJson, options={'style':line_style},),
                 dl.GeoJSON(data = simulatedBldgsJson,options = {'style':style_handle, 'onEachFeature':ns("bindPopup")}, id='simulatedBldgs',hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="status")),colorbar],center=[32.0272,34.7444], zoom=16, style={'width': '100%', 'height': map_height})

if USECOLAB:
    #url = 'https://shai2u.github.io/demographic_estimation_dashboard_article/assets/popup.js'
    #r = requests.get(url, allow_redirects=True)
    #open('/content/assets/popup.js', 'wb').write(r.content)
    !wget https://shai2u.github.io/demographic_estimation_dashboard_article/assets/popup.js
    !mkdir assets
    !mv popup.js assets/

# map3d = html.Iframe(id='ifame-cell', height=map_height, width="100%",
#                                                        src="https://technion-gis.maps.arcgis.com/apps/instant/3dviewer/index.html?appid=73d48240324e45eea139ea9e0c2fe6e1")

d = pd.to_datetime('2015-01-01')
bldgs1 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
bldg_status = bldgs1['status'].value_counts().to_frame().reset_index()
bldg_status.rename(columns={'index':'status','status':'count'},inplace=True)

if len(bldg_status)<3:
  if len(bldg_status[bldg_status['status'].isin(['Building before'])])==0:
    bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building before'],'count':[0]})]).reset_index(drop=True)
  if len(bldg_status[bldg_status['status'].isin(['Construction'])])==0:
    bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Construction'],'count':[0]})]).reset_index(drop=True)
  if len(bldg_status[bldg_status['status'].isin(['Building after'])])==0:
    bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building after'],'count':[0]})]).reset_index(drop=True)
    
bldg_status_count = get_status_graph(bldg_status)


#create an empty dataset with agent coutn
num_of_agents = 3481
MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
#Number of Columns choose:
numOfColumns=59
numOfRows = 59
# set the columns to display the column number
MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
#prepare the y column (row number)
y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
temp = []
for i in range(0,numOfRows):
    temp +=y[i]
y = temp
MatrixPlot['y'] = y

agents_time_new_Comers_stay = agents_stat_summary_by_year[['year','New Comers','stay']].fillna(0)
agents_time_new_Comers_stay['emptyPalces'] = num_of_agents -(agents_time_new_Comers_stay['stay'] +agents_time_new_Comers_stay['New Comers'])


dashboard_page =  html.Div([
  html.Div([
            html.Div([
    html.Div([
            
              #To Do Add static legend, that won't chnaged based on years and statistical stats
                  dbc.Card(
        children=[
            dbc.CardHeader(["Map of the study area ",html.Span(id='3d_map_date')]),
            dbc.CardBody([html.Div(mapObj,id='map_2d',style={'display':'block','textAlign':'center'}),
                        html.Div(
                            []
                            ,id='map_3d')])]),
                  
              # html.Div([
              
              # ])
            ],style={'width': '34%', 'display': 'inline-block','textAlign':'center'}),
        
        html.Div([
                  dbc.Card(
        children=[
            dbc.CardHeader("Time series graph with context"),
            dbc.CardBody(html.Div([dcc.Graph(id='time_seiries_graph',figure=renters_owners_fig,style={'textAlign':'center'})]))])
                  ,

             html.Div([
                                          dbc.Card(
        children=[
            dbc.CardHeader("Current construction typologies"),
            dbc.CardBody([
                         dcc.Graph(id='typo_count', figure=construction_typo_graph,style={'textAlign':'center'}),
                          
            ])])],style={'width': '50%', 'display': 'inline-block','textAlign':'center'}),
              html.Div([
                        dbc.Card(
        children=[ dbc.CardHeader("Building status count"),
            dbc.CardBody([dcc.Graph(id='status_count',figure=bldg_status_count,style={'textAlign':'center'})]
                         )])
                        ],style={'width': '50%', 'display': 'inline-block'})
            ], style={'width': '66%', 'float': 'right', 'display': 'inline-block','textAlign':'center'}),
        ],style={'padding':'1px'}), # seperator between uppwer and lower!
        #-------------------------------- Lower Div --------------------------------------
        html.Div([
                  #---------------------- 2 Variables graph -------------------------------
                  html.Div([
                            html.Div([
                                      dbc.Card(
        children=[
            dbc.CardHeader("2 Variables bubble graph"),
            dbc.CardBody( dcc.Graph(id='bubble_graph',figure=bubble_age_income,style={'width': '100%','marginLeft':'auto','marginRight':'auto','textAlign':'center'}))])])],style={'width': '34%', 'display': 'inline-block','marginLeft':'auto','marginRight':'auto','textAlign':'center'}),
                      html.Div([
                                #To Do Add static legend, that won't chnaged based on years and statistical stats
                    #---------------------- sunburst -------------------------------

                                dbc.Card(
        children=[
            dbc.CardHeader("Population sunburst"),
            dbc.CardBody(dcc.Graph(id='population_sunburst_fig',figure= population_sunburst_graph_init,style={'width': '100%','marginLeft':'auto','marginRight':'auto','textAlign':'center'}))])],style={'width': '33%', 'display': 'inline-block','marginLeft':'auto','marginRight':'auto','textAlign':'center'}),
                      html.Div([
                                
                                #To Do Add static legend, that won't chnaged based on years and statistical stats
                      #---------------------- Dot Matrix -------------------------------

                                dbc.Card(
        children=[
            dbc.CardHeader("Population dot matrix"),
            dbc.CardBody(dcc.Graph(id='dot_matrix_fig',figure=dotMAtrixFig1,style={'width': '100%','marginLeft':'auto','marginRight':'auto','textAlign':'center'}))])],style={'width': '33%', 'display': 'inline-block','marginLeft':'auto','marginRight':'auto','textAlign':'center'})
            ],style={'padding':'1px'}),
                                  #---------------------- Controllers -------------------------------

            html.Div([dbc.Card(
        children=[
            dbc.CardHeader("Controller"),
            dbc.CardBody([html.Div(
                      daq.BooleanSwitch(
                            on=False,
                            label="3D",
                            labelPosition="top",id='3d_map_switch'),style={'width':'10%', 'display': 'inline-block'}),
                    html.Div([dcc.Dropdown( id='select_context',options=[
                                                                             {'label': 'Owners Renters Count', 'value': 'Owners Renters Count'},
                                                                             {'label': 'Chnage Apartment Size', 'value': 'Chnage Apartment Size'},
                                                                             {'label': 'Change in Age Distribution', 'value': 'Change in Age Distribution'},
                                                                             {'label': 'Change in Income Distribution', 'value': 'Change in Income Distribution'},
                                                                             {'label': 'Change in Income Category', 'value': 'Change in Income Category'},



                                  ], value='Owners Renters Count')],style={'width':'25%', 'display': 'inline-block'}),
                  html.Div([dcc.RangeSlider(id='years-slider',
             min=2015,
             max=2030,
             value=[2015,2015],
             marks={str(year):str(year) for year in np.arange(2015,2030,1)},
                                                    step=0.5
                                                    )],style={'width':'65%', 'display': 'inline-block'})
                          ]),


                    
                  ])],style={'textAlign':'center'})
  ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'})
])
external_stylesheets = [dbc.themes.BOOTSTRAP, "https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/style.css"]
app = JupyterDash(__name__,suppress_callback_exceptions=True,prevent_initial_callbacks=True, external_stylesheets=external_stylesheets)


app.title = "Population Dashboard"

#run_with_ngrok(app)

app.layout = html.Div([

    dashboard_page
])

@app.callback(
    Output('simulatedBldgs', 'data'),
    Output('status_count','figure'),
    Output('dot_matrix_fig','figure'),
    Output('time_seiries_graph','figure'),
    Output('typo_count','figure'),
    Output('population_sunburst_fig','figure'),
    Output('bubble_graph','figure'),
    Output('map_2d','style'),
    Output('map_3d','children'),
    Output('3d_map_date','children'),
    Input('years-slider', 'value'),
    Input('select_context','value'),
    Input('3d_map_switch','on')
    # Output('test','children'),
    #Output('dot_matrix_fig','figure'),

)
def update_output_div(input_value,input_select_context,d3_map_switch):
    #simulatedBldgsGdf
    ref_year = int(input_value[0])
    selected_year = int(input_value[1])
    date_return = f'Q3 {selected_year}'
    q_date_for_dot_matrix = f'{selected_year} Q3'
    d = pd.to_datetime(f'{selected_year}-07-01')
    map_libre_date = int(f'{selected_year}0701')
    if input_value[1] % 1 == 0:
        d = pd.to_datetime(f'{selected_year}-01-01')
        date_return = f'Q1 {selected_year}'
        q_date_for_dot_matrix = f'{selected_year} Q1'
        map_libre_date = int(f'{selected_year}0101')
    #reference date
    d_ref = pd.to_datetime(f'{ref_year}-07-01')
    ref_q_date = f'Q3 {ref_year}'
    ref_q_date_e = f'{ref_year} Q3'
    if input_value[0] % 1 == 0:
          d_ref = pd.to_datetime(f'{ref_year}-01-01')
          ref_q_date = f'Q1 {ref_year}'
          ref_q_date_e = f'{ref_year} Q1'

    date_for_js_site = q_date_for_dot_matrix.replace(' ','_')

    bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()
    bldgs['start_date'] = bldgs['start_date'].astype(str)
    bldgs['end_date'] = bldgs['end_date'].astype(str)

    if d3_map_switch:
      bldgsJson=None
      #display 2D/3D Map
      map_2d_display = {'display':'none'}
      #item = gis.content.get("732b15ecf0f848df9ab4d8c8f059ea56")
      #layer = item.layers[0]
      #new_def = {"DefinitionQuery" :f"start_date_int<= {arcgis_date}"}
      #layer.manager.update_definition(new_def)
      rand_id = random_int = np.random.randint(0,10) #to refresh 3d
      # map3d = html.Iframe(id=f'ifame-cell-{rand_id}', height=map_height, width="100%",
      #                                                  src="https://technion-gis.maps.arcgis.com/apps/instant/3dviewer/index.html?appid=73d48240324e45eea139ea9e0c2fe6e1")
      #map_3d_display = {'display':'block'}
      # #/content/scene_batyam.html
      # map3d = html.Iframe(id=f'ifame-cell-{rand_id}', height=map_height, width="100%",
      #                                             src=f"https://shai2u.github.io/demographic_estimation_dashboard_article/dashboard/js_maps/scene_batyam_{date_for_js_site}.html")
      map3d = html.Iframe(id=f'ifame-cell-{rand_id}', height=map_height, width="100%",
                                                  src=f"https://shai2u.github.io/demographic_estimation_dashboard_article/BatYam_maplibre/{map_libre_date}.html")

      #map_3d_display = {'display':'block'}
    else:
      bldgsJson = json.loads(bldgs.to_json())
      #display 2D/3D Map
      map_2d_display = {'display':'block'}
      map3d = ''
      #map_3d_display = {'display':'none'}

    #construction
    bldg_status = bldgs1['status'].value_counts().to_frame().reset_index()
    bldg_status.rename(columns={'index':'status','status':'count'},inplace=True)
    if len(bldg_status)<3:
      if len(bldg_status[bldg_status['status'].isin(['Building before'])])==0:
        bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building before'],'count':[0]})]).reset_index(drop=True)
      if len(bldg_status[bldg_status['status'].isin(['Construction'])])==0:
        bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Construction'],'count':[0]})]).reset_index(drop=True)
      if len(bldg_status[bldg_status['status'].isin(['Building after'])])==0:
        bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building after'],'count':[0]})]).reset_index(drop=True)
    bldg_status_count = get_status_graph(bldg_status)
    
    d2 = d - datetime.timedelta(days=180)
    bldgs2 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)

    bldgs_constr = bldgs2[bldgs2['status']=='Construction'].reset_index()

    construction_typo_v = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
    construction_typo_v.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)

    bldgs3 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d2 ) & (simulatedBldgsGdf['end_date']> d2 )].copy().reset_index(drop=True)
    bldgs_constr = bldgs3[bldgs3['status']=='Construction'].reset_index()

    construction_typo_d = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
    construction_typo_d.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
    construction_typo_graph = construction_typo_graph_generator(construction_typo_v,construction_typo_d)
    #bldg_status_count = get_status_graph(bldg_status)


    # Dot Matrix

    dotMAtrixFig1 = prepare_dot_matrix(q_date_for_dot_matrix)

    #time context
    #renters Owners
    if input_select_context == 'Owners Renters Count':
      context_fig = renters_owners_graph(q_date_for_dot_matrix,ref_q_date_e)
    elif input_select_context == 'Chnage Apartment Size':
      context_fig = apartment_graph(q_date_for_dot_matrix,ref_q_date_e)
    elif input_select_context == 'Change in Age Distribution':
      context_fig = change_age_distribution_graph(q_date_for_dot_matrix,ref_q_date_e)                                                                             
    elif input_select_context == 'Change in Income Distribution':
      context_fig = income_distribution_graph(q_date_for_dot_matrix,ref_q_date_e)
    elif input_select_context == 'Change in Income Category':
      context_fig = income_category_graph(q_date_for_dot_matrix,ref_q_date_e)                                                                           
    else:
      context_fig = renters_owners_graph(q_date_for_dot_matrix,ref_q_date_e)

    #sunburst

    bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()
    agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
    agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

    age_grown1 = d.year-years_with_q2_makrs[0].year
    agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
    agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

    agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
    agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
    population_sunburst_graph = demographic_sunburst(date_return,agents_stay_age_income,colorDict)


    #bubble graphs Selected
    bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()

    agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
    agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

    #age the agents
    age_grown1 = d.year-years_with_q2_makrs[0].year
    agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
    agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

    #set categories to agents ages
    agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
    agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
    #population_sunburst_graph = demographic_sunburst(2019,agents_stay_age_income,colorDict)
    agents_stay_age_income['units'] = 1
    agents_stay_age_income_count = agents_stay_age_income.groupby(['Stay or leave','Age group','Income category']).agg({'units':'count'}).reset_index()
    new_comers_age_income_count = agents_stay_age_income_count[agents_stay_age_income_count['Stay or leave']=='New Comers'].reset_index(drop=True)
    stay_age_income_count = agents_stay_age_income_count[agents_stay_age_income_count['Stay or leave']=='stay'].reset_index(drop=True)
    #total = new_comers_age_income_count['units'].sum()+ stay_age_income_count['units'].sum()
    new_comers_age_income_count['ratio'] = new_comers_age_income_count['units']/1100
    stay_age_income_count['ratio'] = stay_age_income_count['units']/1100

    #reference selection
    bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d_ref ) & (simulatedBldgsGdf['end_date']> d_ref )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()

    agents_synced_buildings_ref_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
    agents_synced_buildings_ref_date_stay_new = agents_synced_buildings_ref_date[agents_synced_buildings_ref_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

    #age the agents
    age_grown1 = d_ref.year-years_with_q2_makrs[0].year
    agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='stay','age'] + age_grown1
    agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d_ref.year - p['start_date'].year),axis=1)

    #set categories to agents ages
    agents_synced_buildings_ref_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_ref_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
    agents_stay_age_income_ref =agents_synced_buildings_ref_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
    #population_sunburst_graph = demographic_sunburst(2019,agents_stay_age_income,colorDict)
    agents_stay_age_income_ref['units'] = 1
    agents_stay_age_income_ref = agents_stay_age_income_ref.groupby(['Stay or leave','Age group','Income category']).agg({'units':'count'}).reset_index()
    new_comers_age_income_count_ref = agents_stay_age_income_ref[agents_stay_age_income_ref['Stay or leave']=='New Comers'].reset_index(drop=True)
    stay_age_income_count_ref = agents_stay_age_income_ref[agents_stay_age_income_ref['Stay or leave']=='stay'].reset_index(drop=True)
    #total_ref = new_comers_age_income_count_ref['units'].sum()+ stay_age_income_count_ref['units'].sum()
    new_comers_age_income_count_ref['ratio'] = new_comers_age_income_count_ref['units']/1100
    stay_age_income_count_ref['ratio'] = stay_age_income_count_ref['units']/1100
    bubble_age_income = bubble_age_income_stay_time(stay_age_income_count_ref,new_comers_age_income_count_ref,stay_age_income_count,new_comers_age_income_count,ref_q_date,date_return)

    # f = open("filter_date.js", "a")
    # f.write(f'date_ = "{arcgis_date}"')
    # f.close()

    # get Javascript filename instead
    #https://befused.com/javascript/get-filename-url
    return bldgsJson,bldg_status_count,dotMAtrixFig1,context_fig,construction_typo_graph,population_sunburst_graph,bubble_age_income,map_2d_display,map3d,q_date_for_dot_matrix

## To dos
## Add Tama 38 Construction Current Coutner

app.run_server(mode='External',host='0.0.0.0',debug=False,port=8050)

FileNotFoundError: [Errno 2] No such file or directory: '/content/assets/popup.js'

In [ ]:
app.logger

<Logger __main__ (INFO)>

In [ ]:
import socket
host = socket.gethostbyname(socket.gethostname())

In [ ]:
host

'172.28.0.2'

In [ ]:

fig = px.colors.qualitative.swatches()
fig.show()

In [ ]:
px.colors.qualitative.Bold_r

['rgb(165, 170, 153)',
 'rgb(249, 123, 114)',
 'rgb(207, 28, 144)',
 'rgb(0, 134, 149)',
 'rgb(230, 131, 16)',
 'rgb(128, 186, 90)',
 'rgb(231, 63, 116)',
 'rgb(242, 183, 1)',
 'rgb(57, 105, 172)',
 'rgb(17, 165, 121)',
 'rgb(127, 60, 141)']

In [ ]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 4.2 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=c979f7413aed8688ae8345731b45bdfde2797afb1b47bc66c7ab6d50d4e8f3ce
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
from pyngrok import ngrok



In [ ]:
    bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()
    bldgs['start_date'] = bldgs['start_date'].astype(str)
    bldgs['end_date'] = bldgs['end_date'].astype(str)
    bldgsJson = json.loads(bldgs.to_json())

In [ ]:
date_for_js_site = '2015 Q1'.replace(' ','_')

In [ ]:
date_for_js_site

'2015_Q1'

In [ ]:
url = 'https://shai2u.github.io/demographic_estimation_dashboard_article/BatYam_maplibre/map.html'
r = requests.get(url, allow_redirects=True)
open('/content/map.html', 'wb').write(r.content)


8375

# After Dash

In [ ]:
contextual_height_global = 275
apartment_graph('2025 Q3','2015 Q1')

In [ ]:
df_less_columns = agents_stat_summary_by_year[['year','New Comers_apartment_size_q1','New Comers_apartment_size_q2','New Comers_apartment_size_q3','stay_apartment_size_q1','stay_apartment_size_q2','stay_apartment_size_q3']].copy().fillna(0)


In [ ]:
agents_stat_summary_by_year

,Unnamed: 0,year,New Comers_apartment_size_q1,stay_apartment_size_q1,New Comers_apartment_size_q2,stay_apartment_size_q2,New Comers_apartment_size_q3,stay_apartment_size_q3,New Comers_age_q1,stay_age_q1,New Comers_age_q2,stay_age_q2,New Comers_age_q3,stay_age_q3,New Comers_income_q1,stay_income_q1,New Comers_income_q2,stay_income_q2,New Comers_income_q3,stay_income_q3,New Comers_rent,stay_rent,New Comers_own,stay_own,New Comers_income_low,stay_income_low,New Comers_income_medium,stay_income_medium,New Comers_income_high,stay_income_high,New Comers,stay,total_pop,New Comers_income_low_ratio,New Comers_income_medium_ratio,New Comers_income_high_ratio,stay_income_low_ratio,stay_income_medium_ratio,stay_income_high_ratio
0,0,2015 Q1,NaN,66.0,NaN,73.0,NaN,83.0,NaN,30.00,NaN,42.0,NaN,65.00,NaN,6403.00,NaN,8771.0,NaN,11667.00,NaN,318,NaN,579,NaN,470.0,NaN,427.0,NaN,0.0,NaN,897.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2015 Q3,NaN,68.0,NaN,73.0,NaN,83.0,NaN,30.00,NaN,42.0,NaN,65.00,NaN,6413.50,NaN,8804.0,NaN,11669.00,NaN,310,NaN,565,NaN,454.0,NaN,421.0,NaN,0.0,NaN,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2016 Q1,NaN,68.0,NaN,73.0,NaN,83.0,NaN,31.00,NaN,43.0,NaN,66.00,NaN,6413.50,NaN,8804.0,NaN,11669.00,NaN,310,NaN,565,NaN,454.0,NaN,421.0,NaN,0.0,NaN,875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2016 Q3,NaN,66.0,NaN,73.0,NaN,83.0,NaN,31.00,NaN,43.0,NaN,65.25,NaN,6427.00,NaN,8833.5,NaN,11729.75,NaN,294,NaN,542,NaN,432.0,NaN,404.0,NaN,0.0,NaN,836.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2017 Q1,82.0,66.0,82.0,73.0,82.0,83.0,32.00,32.00,47.0,44.0,60.00,66.75,18838.105263,6496.00,21097.105263,8934.5,22033.947368,11806.25,11.0,282,14.0,520,0.0,405.0,11.0,397.0,14.0,0.0,25.0,802.0,827.0,0.0,0.013301,0.016929,0.489722,0.480048,0.0
5,0,2017 Q3,82.0,66.0,82.0,73.0,82.0,83.0,32.00,32.00,47.0,44.0,60.00,66.75,18838.105263,6496.00,21097.105263,8934.5,22033.947368,11806.25,11.0,282,14.0,520,0.0,405.0,11.0,397.0,14.0,0.0,25.0,802.0,827.0,0.0,0.013301,0.016929,0.489722,0.480048,0.0
6,0,2018 Q1,82.0,66.0,82.0,73.0,92.0,83.0,31.75,33.00,42.0,45.0,60.25,68.00,19619.105263,6497.00,21099.868421,8933.0,22017.447368,11867.50,31.0,278,53.0,517,0.0,402.0,20.0,393.0,64.0,0.0,84.0,795.0,879.0,0.0,0.022753,0.072810,0.457338,0.447099,0.0
7,0,2018 Q3,72.0,66.0,82.0,73.0,92.0,83.0,31.00,33.00,44.0,45.0,59.00,68.00,18584.605263,6497.00,20230.421053,8933.0,22263.842105,11867.50,69.0,278,134.0,517,0.0,402.0,75.0,393.0,128.0,0.0,203.0,795.0,998.0,0.0,0.075150,0.128257,0.402806,0.393788,0.0
8,0,2019 Q1,82.0,66.0,82.0,73.0,102.0,80.0,32.00,34.00,46.0,46.5,62.00,69.00,19415.105263,6503.75,21815.473684,8965.0,25173.131579,11886.75,90.0,271,195.0,493,0.0,384.0,75.0,380.0,210.0,0.0,285.0,764.0,1049.0,0.0,0.071497,0.200191,0.366063,0.362250,0.0
9,0,2019 Q3,82.0,66.0,82.0,73.0,102.0,80.0,32.00,34.00,46.0,46.5,62.00,69.00,19415.105263,6503.75,21815.473684,8965.0,25173.131579,11886.75,90.0,271,195.0,493,0.0,384.0,75.0,380.0,210.0,0.0,285.0,764.0,1049.0,0.0,0.071497,0.200191,0.366063,0.362250,0.0


In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_apartment_size_q1'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Apartment Size Q1",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_apartment_size_q2'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Apartment Size Q2",
    mode="lines",
    line=dict(color='royalblue', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_apartment_size_q3'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Apartment Size Q3",
    mode="lines",
    line=dict(color='royalblue', width=4)
    
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_apartment_size_q1'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Apartment Size Q1",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_apartment_size_q2'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Apartment Size Q2",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_apartment_size_q3'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Apartment Size Q3",
    mode="lines",
    line=dict(color='firebrick', width=4)
    
))

fig.update_layout(title="Apartment Size  (q1-q3) vs Staying or New Comers",template='plotly_white',yaxis = {'title' : "m<sup>2</sup>"})

fig.show()

# set range

In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_age_q1'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Age Q1",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_age_q2'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Age Q2",
    mode="lines",
    line=dict(color='royalblue', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_age_q3'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Age Q3",
    mode="lines",
    line=dict(color='royalblue', width=4)
    
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_age_q1'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Age Q1",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_age_q2'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Age Q2",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_age_q3'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Age Q3",
    mode="lines",
    line=dict(color='firebrick', width=4)
    
))


fig.update_layout(title="age (q1,q2,q3) of olderst person at household",template='plotly_white',yaxis = {'title' : "Average Age"})

fig.show()

# set range

In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
def change_age_distribution_graph(year_q,q_date):
  df_less_columns = agents_stat_summary_by_year[['year','New Comers_age_q1','New Comers_age_q2','New Comers_age_q3','stay_age_q1','stay_age_q2','stay_age_q3']].copy().fillna(0)
  end_index = df_less_columns[df_less_columns['year']==year_q].index.values[0]
  selected_indexes = range(0,end_index+1)
  df_for_graph = df_less_columns[df_less_columns.index.isin(selected_indexes)]

  fig = go.Figure()


  # start modification


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_age_q1'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Average age in q1 (25%)",
      mode="lines",
      line=dict(color='royalblue', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_age_q2'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Average age in q2 (50%)",
      mode="lines",
      line=dict(color='royalblue', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['New Comers_age_q3'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Average age in q3 (75%)",
      mode="lines",
      line=dict(color='royalblue', width=4)
      
  ))

  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_age_q1'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Average age in q1 (25%)",
      mode="lines",
      line=dict(color='firebrick', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_age_q2'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Average age in q2 (50%)",
      mode="lines",
      line=dict(color='firebrick', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=year_ranges,
      y=df_for_graph['stay_age_q3'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Average age in q2 (75%)",
      mode="lines",
      line=dict(color='firebrick', width=4)
      
  ))

  

  fig.add_trace(go.Scatter(x=[q_date, q_date], y=[20,100],
              name=q_date,
              legendgroup="Reference",
              legendgrouptitle_text="Reference",
              mode="lines",
              line=dict(color="LightSeaGreen", width=2,dash="dashdot")))
  
  fig.update_layout(title="Age quarter distribution for Staying vs New Comers",template='plotly_white',yaxis = {'title' : "Average Age"},margin={"r":0,"t":35,"l":0,"b":0,"pad":0},
                    width=contextual_width_global,height=contextual_height_global,legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))



  fig.update_xaxes(range = [0,len(year_ranges)])
  fig.update_yaxes(range = [20,110])
  #end modification

  return fig

# set range
graph = change_age_distribution_graph('2025 Q1','2015 Q1')

In [ ]:
graph

In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()



fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_q1'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Income Q1",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_q2'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Income Q2",
    mode="lines",
    line=dict(color='royalblue', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_q3'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Income Q3",
    mode="lines",
    line=dict(color='royalblue', width=4)
    
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_q1'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Icome Q1",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_q2'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Income Q2",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_q3'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Income Q3",
    mode="lines",
    line=dict(color='firebrick', width=4)
    
))

fig.update_layout(title="Income (q1-q3) of olderst person at household",template='plotly_white',yaxis = {'title' : "Inocme (New Israeli Shekels)"})

fig.show()

# set range

In [ ]:
graph

In [ ]:
rent_own_new_stay = agents_stat_summary_by_year[['year','New Comers_rent','New Comers_own','stay_rent','stay_own']].copy().fillna(0)
end_index = rent_own_new_stay[rent_own_new_stay['year']=='2019 Q1'].index.values[0]
selected_indexes = range(0,end_index+1)
rent_own_new_stayselected = rent_own_new_stay[rent_own_new_stay.index.isin(selected_indexes)]

In [ ]:
rent_own_new_stayselected

,year,New Comers_rent,New Comers_own,stay_rent,stay_own
0,2015 Q1,0.0,0.0,318,579
1,2015 Q3,0.0,0.0,310,565
2,2016 Q1,0.0,0.0,310,565
3,2016 Q3,0.0,0.0,294,542
4,2017 Q1,11.0,14.0,282,520
5,2017 Q3,11.0,14.0,282,520
6,2018 Q1,31.0,53.0,278,517
7,2018 Q3,69.0,134.0,278,517
8,2019 Q1,90.0,195.0,271,493


In [ ]:
agents_stat_summary_by_yearæ

NameError: ignored

In [ ]:
fig

Their is a lot of ownown regrading how the population will look like because this is a complex matter

In [ ]:

agents_stat_summary_by_year['New Comers'] = agents_stat_summary_by_year['New Comers_income_low'] + agents_stat_summary_by_year['New Comers_income_medium'] +  + agents_stat_summary_by_year['New Comers_income_high']
agents_stat_summary_by_year['stay'] = agents_stat_summary_by_year['stay_income_low'] + agents_stat_summary_by_year['stay_income_medium'] +  + agents_stat_summary_by_year['stay_income_high']
agents_stat_summary_by_year['total_pop'] = agents_stat_summary_by_year['New Comers'] + agents_stat_summary_by_year['stay']
agents_stat_summary_by_year['New Comers_income_low_ratio'] = agents_stat_summary_by_year['New Comers_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_medium_ratio'] = agents_stat_summary_by_year['New Comers_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['New Comers_income_high_ratio'] = agents_stat_summary_by_year['New Comers_income_high']/agents_stat_summary_by_year['total_pop']

agents_stat_summary_by_year['stay_income_low_ratio'] = agents_stat_summary_by_year['stay_income_low']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_medium_ratio'] = agents_stat_summary_by_year['stay_income_medium']/agents_stat_summary_by_year['total_pop']
agents_stat_summary_by_year['stay_income_high_ratio'] = agents_stat_summary_by_year['stay_income_high']/agents_stat_summary_by_year['total_pop']


In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_low_ratio'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Low Inocme",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_medium_ratio'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="Medium Income",
    mode="lines",
    line=dict(color='royalblue', width=3)
))

fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers_income_high_ratio'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="High Income",
    mode="lines",
    line=dict(color='royalblue', width=5)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_low_ratio'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Renters",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_medium_ratio'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Medium",
    mode="lines",
    line=dict(color='firebrick', width=3)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay_income_high_ratio'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="High",
    mode="lines",
    line=dict(color='firebrick', width=5)
))


fig.update_layout(title="Income by class",template='plotly_white',yaxis = {'title' : "ratio"})

fig.show()

# set range

In [ ]:
graph

In [ ]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['New Comers'],
    legendgroup="New Comers",  # this can be any string, not just "group"
    legendgrouptitle_text="New Comers",
    name="New comers",
    mode="lines",
    line=dict(color='royalblue', width=1)
))


fig.add_trace(go.Scatter(
    x=year_ranges,
    y=agents_stat_summary_by_year['stay'],
    legendgroup="Staying",  # this can be any string, not just "group"
    legendgrouptitle_text="Staying",
    name="Staying",
    mode="lines",
    line=dict(color='firebrick', width=1)
))


fig.update_layout(title="Number of People",template='plotly_white',yaxis = {'title' : "Absolute"})

fig.show()

# set range

# Building Typology

In [ ]:
# Generate count status graph
def get_construction_typo_graph(construction_typo):
  fig = px.bar(bldg_status, x="status", y="count",width=600,height=250,template='plotly_white',title="Current Building Status",category_orders={'status':['Building before', 'Construction', 'Building after']})
  fig.update_layout(margin={"r":0,"t":35,"l":0,"b":0,"pad":0},
  showlegend=True,

  # font=dict(
  #     size=15,
  # )
  )
  fig.update_yaxes(automargin=True)
  fig.update_yaxes(range=[0, 70])
  return fig

In [ ]:
d1 = pd.to_datetime('2022-01-01')
d2 = d1 - datetime.timedelta(days=180)
bldgs1 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d1 ) & (simulatedBldgsGdf['end_date']> d1 )].copy().reset_index(drop=True)
construction_typo_v = bldgs1['project_ty'].value_counts().to_frame().reset_index()
construction_typo_v.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
bldgs1 = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d2 ) & (simulatedBldgsGdf['end_date']> d2 )].copy().reset_index(drop=True)
construction_typo_d = bldgs1['project_ty'].value_counts().to_frame().reset_index()
construction_typo_d.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
#bldg_status_count = get_status_graph(bldg_status)

In [ ]:
construction_typo

NameError: ignored

In [ ]:
v_a = construction_typo[construction_typo['project_ty']==1]['count'].values[0]
v_r = construction_typo[construction_typo['project_ty']==2]['count'].values[0]
v_rr = construction_typo[construction_typo['project_ty']==2]['count'].values[0]

In [ ]:
d = pd.to_datetime('2025-01-01')
bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< d ) & (simulatedBldgsGdf['end_date']> d )].copy().reset_index(drop=True)
bldgs1 = bldgs.copy()
agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

In [ ]:
agents_synced_buildings_to_date_stay_new['status_x']

0             stay
1             stay
2             stay
3             stay
4             stay
           ...    
1921    New Comers
1922    New Comers
1923    New Comers
1924    New Comers
1925    New Comers
Name: status_x, Length: 1926, dtype: object

In [ ]:
year_makrs = [year for year in np.arange(2015,2031,0.5)]
years_with_q2_makrs = []
for year in year_makrs:
    d = pd.to_datetime(f'{int(year)}-07-01')
    if year % 1 == 0:
        d = pd.to_datetime(f'{int(year)}-01-01')
    years_with_q2_makrs.append(d)

In [ ]:
age_grown1 = d.year-years_with_q2_makrs[0].year
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})

In [ ]:
agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']]

,status_x,age_group,income_cat
0,stay,45-64,Medium
1,stay,45-64,Medium
2,stay,45-64,Medium
3,stay,45-64,Medium
4,stay,45-64,Medium
...,...,...,...
1921,New Comers,45-64,High
1922,New Comers,45-64,High
1923,New Comers,45-64,High
1924,New Comers,85+,High


In [ ]:
fig = px.sunburst(agents_stay_age_income, path=['Stay or leave','Age group','Income category'])

In [ ]:
fig

In [ ]:
#https://github.com/Shai2u/demographic_estimation_dashboard_article/blob/main/assets/color_labels.xlsx
color_labels = pd.read_excel(
    'https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/color_labels.xlsx')
colorDict = dict(zip(color_labels['label'], color_labels['colors_']))


In [ ]:
colorDict

{'18-44': 'hsv(205,80%,80%)',
 '45-64': 'hsv(205,60%,90%)',
 '65-84': 'hsv(205,30%,90%)',
 '85+': 'hsv(205,10%,90%)',
 'High': 'hsv(350,90%,75%)',
 'Low': 'hsv(350,25%,100%)',
 'Medium': 'hsv(350,50%,95%)',
 'New Comers': 'hsv(210,90%,50%)',
 'stay': 'hsv(350,90%,50%)'}

In [ ]:
d

Timestamp('2030-07-01 00:00:00')

In [ ]:
labels_text = fig.data[0].labels.tolist()

In [ ]:
labels_text

['Low',
 'Low',
 'Low',
 'Low',
 'Low',
 'Low',
 'Low',
 'Low',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 'High',
 '18-44',
 '18-44',
 '45-64',
 '45-64',
 '65-84',
 '65-84',
 '85+',
 '85+',
 'New Comers',
 'stay']

In [ ]:
fig.data[0].marker.colors

# Sunburst

In [ ]:
    def sunburst_graph(r, year_, color_field='raw age', colorDict_=colorDict, group_color_dict=group_color_dict):
        title_ = f'{year_} : Market Vs Affordable Units Age/Income in WIRE'
        # color_discrete_map = color_group_map_
        colors_ = group_color_dict['colors_age'].unique().tolist()
        fig = px.sunburst(r, path=['ap_class', 'Age Group', 'Income'],
                          color=color_field, color_continuous_scale=colors_, title=title_,)

        labels_text = fig.data[0].labels.tolist()
        colorLabels = tuple(colorDict_[item] for item in labels_text)
        fig.data[0].marker.colors = colorLabels
        fig.update_traces(textinfo="label+percent entry")
        fig.update_layout(showlegend=False, margin=dict(l=50, r=50, t=100, b=50), legend=dict(
            yanchor="top", y=1, xanchor="left", x=1, orientation="h"), width=sim_plot.cont_width, height=sim_plot.tl_height+50,font=dict(size=sim_plot.textSize_))
        return fig

NameError: ignored

# Bubble Figure

In [ ]:
    def bubbleAgeIncomeClass(r, year_, titleText_):
        r2 = r.groupby(['ap_class', 'group_name', 'Age Group', 'Income']).agg(
            {'raw age': 'count'}).reset_index().rename(columns={'raw age': 'count', 'ap_class': 'Ap Type'})

        title_ = str(year_)+' '+titleText_ + ' Age/Income'

        fig = px.scatter(r2, x="Age Group", y="Income",
                         size="count", color="group_name", color_discrete_map=colorDictMerge, facet_col='Ap Type', title=title_, size_max=30,
                         category_orders={"Age Group": ["18-44", "45-64", "65-84", "85+"],
                                          "Income": ['Upper', 'Middle', 'Moderate', 'Low']}, template='ggplot2')
        
        fig.update_layout(showlegend=False, margin=dict(l=50, r=50, t=100, b=50), width=sim_plot.cont_width, height=sim_plot.tl_height,font=dict(size=sim_plot.textSize_))

        return fig

In [ ]:
fig = make_subplots(rows=1, cols=2,subplot_titles=("Reference Date", "Current Date"))

fig.add_trace(
    go.Scatter(x=[1, 2, 3], y=[4, 5, 6], mode="markers",name='Reference Date',marker=dict(size=20,color='green')),
    row=1, col=1)

fig.add_trace(
    go.Scatter(x=[20, 30, 40], y=[50, 60, 70], mode="markers",name='Current Date'),
    row=1, col=2
)


fig.update_layout(height=600, width=800, title_text="Side By Side Subplots", margin=dict(l=0, r=0, t=50, b=0),legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01),template='plotly_white')
fig.show()

# 3D Controller!

In [ ]:
from arcgis.gis import GIS

In [ ]:
#from arcgis.gis import GIS

#from arcgis import GIS
#gis = GIS("home")
#gis = GIS("https://technion-gis.maps.arcgis.com", "shai@technion")
gis = GIS("https://technion-gis.maps.arcgis.com",client_id='wI3SgTPXnv2N3B4P')
item = gis.content.get("732b15ecf0f848df9ab4d8c8f059ea56")
new_def = {"DefinitionQuery" :"start_date_int<= 205011"}
layer.manager.update_definition(new_def)


Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://technion-gis.maps.arcgis.com/sharing/rest//oauth2/authorize?response_type=code&client_id=wI3SgTPXnv2N3B4P&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=h1689z5AgHHiGE8mnSgfWcsjd6FM2y
Enter code obtained on signing in using SAML: ··········


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
gis = GIS("https://technion-gis.maps.arcgis.com",client_id='wI3SgTPXnv2N3B4P')
item = gis.content.get("732b15ecf0f848df9ab4d8c8f059ea56")
new_def = {"DefinitionQuery" :""}
layer = item.layers[0]
layer.manager.update_definition(new_def)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://technion-gis.maps.arcgis.com/sharing/rest//oauth2/authorize?response_type=code&client_id=wI3SgTPXnv2N3B4P&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=n64BpeogNCRUBU5RXtQZPR600phQ6p
Enter code obtained on signing in using SAML: ··········


/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning:

Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



{'success': True}

In [ ]:
layer =item.layers[0]

In [ ]:
layer

<FeatureLayer url:"https://services1.arcgis.com/yAQXemoDSgzdfV2A/arcgis/rest/services/ts_test_aug_10/FeatureServer/0">

In [ ]:
layer.

<bound method FeatureLayer.query of <FeatureLayer url:"https://services1.arcgis.com/yAQXemoDSgzdfV2A/arcgis/rest/services/ts_test_aug_10/FeatureServer/0">>

In [ ]:
webS = arcgis.mapping.WebScene(item)


In [ ]:
webS['operationalLayers'][3]['layerDefinition']['definitionExpression']

"status IN ('Building after','') OR status IS NULL"

In [ ]:
webS['operationalLayers'][14]['layerDefinition']['definitionExpression']

'(end_date_int > 201412) AND (end_date_int <= 202571)'

In [ ]:
webS['operationalLayers'][14]['layerDefinition']['definitionExpression'] = "((end_date_int >= 201412) AND (end_date_int <= 202571)) AND (status IN ('Building after','') OR status IS NULL)"

In [ ]:
webS['operationalLayers'][14]['layerDefinition']['definitionExpression'] = '"2027-07-01"< end_date_2'

In [ ]:
webS.update()

In [ ]:
d = pd.to_datetime('2027-07-01')

In [ ]:
d

Timestamp('2027-07-01 00:00:00')

In [ ]:
webS['operationalLayers'][14]['layerDefinition']['definitionExpression']

"end_date_2 IN ('2022-09-28','') OR end_date_2 IS NULL"

In [ ]:
d

NameError: ignored

In [ ]:
bbl = "BBL IN ('1013730001','1013737501','1013737502','1013737503','1013737504','1013737505','1013730040','1013730030') OR BBL IS NULL"
webS['operationalLayers'][0]['layerDefinition']['definitionExpression'] = bbl
webS.update()

In [ ]:
['layerDefinition']

In [ ]:

#hovertext

# fig = go.Figure(go.Scatter(
#     x = [1,2,3,4,5],
#     y = [2.02825,1.63728,6.83839,4.8485,4.73463],
#     hovertemplate =
#     '<i>Price</i>: $%{y:.2f}'+
#     '<br><b>X</b>: %{x}<br>'+
#     '<b>%{text}</b>',
#     text = ['Custom text {}'.format(i + 1) for i in range(5)],
#     showlegend = False))

# fig.add_trace(go.Scatter(
#     x = [1,2,3,4,5],
#     y = [3.02825,2.63728,4.83839,3.8485,1.73463],
#     hovertemplate = 'Price: %{y:$.2f}<extra></extra>',
#     showlegend = False))

# fig.update_layout(
#     hoverlabel_align = 'right',
#     title = "Set hover text with hovertemplate")

# fig.show()

In [ ]:
d

Timestamp('2022-01-01 00:00:00')